In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/spaceship-titanic/sample_submission.csv
/kaggle/input/spaceship-titanic/train.csv
/kaggle/input/spaceship-titanic/test.csv


In [2]:
train_df = pd.read_csv("/kaggle/input/spaceship-titanic/train.csv")
test_df = pd.read_csv("/kaggle/input/spaceship-titanic/test.csv")


In [3]:
train_df.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


In [4]:
test_df.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name
0,0013_01,Earth,True,G/3/S,TRAPPIST-1e,27.0,False,0.0,0.0,0.0,0.0,0.0,Nelly Carsoning
1,0018_01,Earth,False,F/4/S,TRAPPIST-1e,19.0,False,0.0,9.0,0.0,2823.0,0.0,Lerome Peckers
2,0019_01,Europa,True,C/0/S,55 Cancri e,31.0,False,0.0,0.0,0.0,0.0,0.0,Sabih Unhearfus
3,0021_01,Europa,False,C/1/S,TRAPPIST-1e,38.0,False,0.0,6652.0,0.0,181.0,585.0,Meratz Caltilter
4,0023_01,Earth,False,F/5/S,TRAPPIST-1e,20.0,False,10.0,0.0,635.0,0.0,0.0,Brence Harperez


In [5]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   8693 non-null   object 
 1   HomePlanet    8492 non-null   object 
 2   CryoSleep     8476 non-null   object 
 3   Cabin         8494 non-null   object 
 4   Destination   8511 non-null   object 
 5   Age           8514 non-null   float64
 6   VIP           8490 non-null   object 
 7   RoomService   8512 non-null   float64
 8   FoodCourt     8510 non-null   float64
 9   ShoppingMall  8485 non-null   float64
 10  Spa           8510 non-null   float64
 11  VRDeck        8505 non-null   float64
 12  Name          8493 non-null   object 
 13  Transported   8693 non-null   bool   
dtypes: bool(1), float64(6), object(7)
memory usage: 891.5+ KB


8693,4277

In [6]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4277 entries, 0 to 4276
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   4277 non-null   object 
 1   HomePlanet    4190 non-null   object 
 2   CryoSleep     4184 non-null   object 
 3   Cabin         4177 non-null   object 
 4   Destination   4185 non-null   object 
 5   Age           4186 non-null   float64
 6   VIP           4184 non-null   object 
 7   RoomService   4195 non-null   float64
 8   FoodCourt     4171 non-null   float64
 9   ShoppingMall  4179 non-null   float64
 10  Spa           4176 non-null   float64
 11  VRDeck        4197 non-null   float64
 12  Name          4183 non-null   object 
dtypes: float64(6), object(7)
memory usage: 434.5+ KB


In [7]:
train_df = train_df.drop(['PassengerId','Name'],axis=1)
test_df = test_df.drop(['Name'],axis=1)

In [8]:
train_df["HomePlanet"].value_counts()

HomePlanet
Earth     4602
Europa    2131
Mars      1759
Name: count, dtype: int64

In [9]:
train_df["HomePlanet"].isnull().sum()

201

In [10]:
train_df['CryoSleep'] = train_df['CryoSleep'].fillna(0)
train_df['VIP'] = train_df['VIP'].fillna(0)
train_df['Cabin'] = train_df['Cabin'].fillna(train_df['Cabin'].mode()[0])
train_df['HomePlanet'] = train_df['HomePlanet'].fillna(train_df['HomePlanet'].mode()[0])
train_df['Destination'] = train_df['Destination'].fillna(train_df['Destination'].mode()[0])
train_df['ShoppingMall'] = train_df['ShoppingMall'].fillna(train_df['ShoppingMall'].median())
train_df['VRDeck'] = train_df['VRDeck'].fillna(train_df['VRDeck'].median())
train_df['FoodCourt'] = train_df['FoodCourt'].fillna(train_df['FoodCourt'].median())
train_df['Spa'] = train_df['Spa'].fillna(train_df['Spa'].median())
train_df['RoomService'] = train_df['RoomService'].fillna(train_df['RoomService'].median())
train_df['Age'] = train_df['Age'].fillna(train_df['Age'].median())

In [11]:
cab = train_df["Cabin"].apply(lambda x: x.split("/"))

train_df["Cab_1"] = cab.apply(lambda x: x[0])
train_df["Cab_3"] = cab.apply(lambda x: x[2])
train_df["Cab_2"] = cab.apply(lambda x: float(x[1]))

In [12]:
train_df = train_df.drop(['Cabin'],axis=1)

In [13]:
train_df["VIP"] = train_df["VIP"].astype(int)
train_df["CryoSleep"] = train_df["CryoSleep"].astype(int)

In [14]:
train_df.isna().sum() 

HomePlanet      0
CryoSleep       0
Destination     0
Age             0
VIP             0
RoomService     0
FoodCourt       0
ShoppingMall    0
Spa             0
VRDeck          0
Transported     0
Cab_1           0
Cab_3           0
Cab_2           0
dtype: int64

In [15]:
input_data = train_df.drop(['Transported'], axis = 1)
encoding_train_data = pd.get_dummies(input_data)
encoding_train_data

,CryoSleep,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Cab_2,HomePlanet_Earth,...,Cab_1_A,Cab_1_B,Cab_1_C,Cab_1_D,Cab_1_E,Cab_1_F,Cab_1_G,Cab_1_T,Cab_3_P,Cab_3_S
0,0,39.0,0,0.0,0.0,0.0,0.0,0.0,0.0,False,...,False,True,False,False,False,False,False,False,True,False
1,0,24.0,0,109.0,9.0,25.0,549.0,44.0,0.0,True,...,False,False,False,False,False,True,False,False,False,True
2,0,58.0,1,43.0,3576.0,0.0,6715.0,49.0,0.0,False,...,True,False,False,False,False,False,False,False,False,True
3,0,33.0,0,0.0,1283.0,371.0,3329.0,193.0,0.0,False,...,True,False,False,False,False,False,False,False,False,True
4,0,16.0,0,303.0,70.0,151.0,565.0,2.0,1.0,True,...,False,False,False,False,False,True,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,0,41.0,1,0.0,6819.0,0.0,1643.0,74.0,98.0,False,...,True,False,False,False,False,False,False,False,True,False
8689,1,18.0,0,0.0,0.0,0.0,0.0,0.0,1499.0,True,...,False,False,False,False,False,False,True,False,False,True
8690,0,26.0,0,0.0,0.0,1872.0,1.0,0.0,1500.0,True,...,False,False,False,False,False,False,True,False,False,True
8691,0,32.0,0,0.0,1049.0,0.0,353.0,3235.0,608.0,False,...,False,False,False,False,True,False,False,False,False,True


In [16]:
y_target = train_df['Transported']
x_data = encoding_train_data.drop(['Cab_1_A', 'Cab_1_G', 'Cab_1_T', 'Cab_1_D', 'Cab_1_C', 'VIP', 'HomePlanet_Mars', 'Destination_PSO J318.5-22'], axis = 1)

pca = PCA(n_components = 10)
pca.fit(encoding_train_data)
pca_train_data = pca.transform(encoding_train_data)
pca_train_data

pca_cols = []
for i in range(0, 10) :
    x = 'pca_col' + str(i)
    pca_cols.append(x)
pca_cols
pca_df = pd.DataFrame(pca_train_data, columns = pca_cols)
pca_df

,pca_col0,pca_col1,pca_col2,pca_col3,pca_col4,pca_col5,pca_col6,pca_col7,pca_col8,pca_col9
0,-535.341608,-188.714234,-27.402263,-245.542583,-117.236821,-653.549626,9.880905,-0.795466,-0.333336,-0.024570
1,-364.459550,174.658510,351.457547,-144.056623,-124.735259,-625.822240,-5.913153,0.531463,-0.167721,-0.964980
2,4601.931282,2538.931628,4854.954707,-308.596940,-180.206149,-199.073765,19.218088,0.580175,-0.077342,-0.557735
3,1609.037457,1476.550534,2291.396683,-211.097397,214.339625,-444.473482,-0.914852,0.608189,-0.336551,-0.338938
4,-318.081789,132.973183,397.172652,73.585942,-48.130304,-620.137253,-14.317681,0.542590,-0.169890,-0.926218
...,...,...,...,...,...,...,...,...,...,...
8688,6177.764461,-1848.039711,1045.851479,-222.926866,-92.463198,-162.508568,5.446957,-0.776062,0.103720,0.706790
8689,-638.376256,-249.958356,-24.085179,-272.003927,-105.952546,840.422038,-7.052930,0.881964,0.700776,0.750458
8690,-646.566241,-232.940637,6.041793,170.495177,1712.649361,835.555114,-0.404533,0.880952,0.697199,-0.144583
8691,1397.579301,1789.264375,-1956.462314,-201.677446,-103.007847,175.840205,1.038971,0.723251,-0.335986,0.873072


In [17]:
x_train, x_test, y_train, y_test = train_test_split(pca_df, y_target, test_size = 0.2, random_state = 0)

In [18]:
pca_model = RandomForestClassifier(n_estimators=100, random_state=5,
                              max_depth=10, min_samples_leaf = 4, min_samples_split = 16, oob_score=True)
pca_model.fit(x_train, y_train)

pred = pca_model.predict(x_test)

# 평가

print("훈련 세트 정확도: {:.3f}".format(pca_model.score(x_train, y_train)) )
print("테스트 세트 정확도: {:.3f}".format(pca_model.score(x_test, y_test)) )
print("OOB 샘플의 정확도: {:.3f}".format(pca_model.oob_score_) )

훈련 세트 정확도: 0.864
테스트 세트 정확도: 0.795
OOB 샘플의 정확도: 0.797


In [19]:
test_df['CryoSleep'] = test_df['CryoSleep'].fillna(0)
test_df['VIP'] = test_df['VIP'].fillna(0)
test_df['Cabin'] = test_df['Cabin'].fillna(test_df['Cabin'].mode()[0])
test_df['HomePlanet'] = test_df['HomePlanet'].fillna(test_df['HomePlanet'].mode()[0])
test_df['Destination'] = test_df['Destination'].fillna(test_df['Destination'].mode()[0])
test_df['ShoppingMall'] = test_df['ShoppingMall'].fillna(test_df['ShoppingMall'].median())
test_df['VRDeck'] = test_df['VRDeck'].fillna(test_df['VRDeck'].median())
test_df['FoodCourt'] = test_df['FoodCourt'].fillna(test_df['FoodCourt'].median())
test_df['Spa'] = test_df['Spa'].fillna(test_df['Spa'].median())
test_df['RoomService'] = test_df['RoomService'].fillna(test_df['RoomService'].median())
test_df['Age'] = test_df['Age'].fillna(test_df['Age'].median())

In [20]:
cab = test_df["Cabin"].apply(lambda x: x.split("/"))

test_df["Cab_1"] = cab.apply(lambda x: x[0])
test_df["Cab_3"] = cab.apply(lambda x: x[2])
test_df["Cab_2"] = cab.apply(lambda x: float(x[1]))

In [21]:
test_df = test_df.drop(['Cabin'],axis=1)

In [22]:
test_df["VIP"] = test_df["VIP"].astype(int)
test_df["CryoSleep"] = test_df["CryoSleep"].astype(int)

In [23]:
encoding_test_data = pd.get_dummies(test_df)
test_x_data = encoding_test_data.drop(['Cab_1_A', 'Cab_1_G', 'Cab_1_T', 'Cab_1_D', 'Cab_1_C', 'VIP', 'HomePlanet_Mars', 'Destination_PSO J318.5-22'], axis = 1)

In [24]:
pca = PCA(n_components = 10)
pca.fit(encoding_test_data)
pca_test_data = pca.transform(encoding_test_data)

pca_cols = []
for i in range(0, 10) :
    x = 'pca_col' + str(i)
    pca_cols.append(x)
pca_df = pd.DataFrame(pca_test_data, columns = pca_cols)

pca_rf_pred = pca_model.predict(pca_df)

sample_submission = pd.read_csv('/kaggle/input/spaceship-titanic/sample_submission.csv')
sample_submission['Transported'] = pca_rf_pred
sample_submission.to_csv('submission.csv',index = False)
sample_submission.head()

,PassengerId,Transported
0,0013_01,False
1,0018_01,False
2,0019_01,False
3,0021_01,True
4,0023_01,False
